<a href="https://colab.research.google.com/github/annab0503/DS2002/blob/main/Project%201/Analysis%20Data/Visa_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import Dependencies
import pandas as pd
import os
import requests

In [ ]:
# Step 1: Load the Excel file from URL
url = 'https://github.com/annab0503/DS4002/blob/main/Project%201/Original%20Data/FYs97-14_NIVDetailTable.xls?raw=true'
response = requests.get(url)
excel_file_path = '/content/FYs97-14_NIVDetailTable.xls'

# Save the file locally
with open(excel_file_path, 'wb') as f:
    f.write(response.content)

# Load the Excel file
excel_data = pd.ExcelFile(excel_file_path)

# Directory to save the CSV files
csv_directory = '/content/csv_files'
os.makedirs(csv_directory, exist_ok=True)  # Ensure the directory exists

# Step 2: Iterate through each sheet in the Excel file and save as CSV
for sheet_name in excel_data.sheet_names:
    # Load the sheet into a DataFrame
    df = excel_data.parse(sheet_name)

    # Define the output CSV file name
    csv_file = os.path.join(csv_directory, f'{sheet_name}.csv')

    # Save the DataFrame to a CSV file
    df.to_csv(csv_file, index=False)
    print(f"Saved sheet '{sheet_name}' to {csv_file}")

# Step 3: Iterate through each CSV file in the directory and process
csv_files = [f for f in os.listdir(csv_directory) if f.endswith('.csv')]

# Loop over each CSV file for processing
for csv_file in csv_files:
    # Load the CSV file
    year_data = pd.read_csv(os.path.join(csv_directory, csv_file))

    # Drop empty rows
    year_data = year_data.dropna(how='all')

    # Dynamically find and rename the 'Fiscal Year' column to 'Country'
    fiscal_year_column = next((col for col in year_data.columns if 'Fiscal Year' in col), None)
    if fiscal_year_column:
        year_data.rename(columns={fiscal_year_column: 'Country'}, inplace=True)

    # Drop unnecessary description rows like continents and totals
    countries_to_drop = ['Africa', 'Asia', 'Europe', 'North America', 'Oceania', 'South America', 'Unknown']
    year_data = year_data[~year_data['Country'].isin(countries_to_drop)]

    # Drop rows containing 'Totals'
    year_data = year_data[~year_data['Country'].str.contains('Totals', na=False)]

    # Reshape the DataFrame with 'melt'
    year_data = year_data.melt(id_vars=['Country'],
                                var_name='Type of U.S. Visa',
                                value_name='Quantity of U.S Visas Granted')

    # Extract the year from the filename (e.g., 'FY1997.csv')
    year = int(csv_file.split('FY')[1].split('.csv')[0])
    year_data['Year'] = year

    # Save the reshaped DataFrame to a new CSV file
    output_file = os.path.join(csv_directory, f"processed_{csv_file}")
    year_data.to_csv(output_file, index=False)

    print(f"Processed {csv_file} and saved to {output_file}")

Saved sheet 'FY97' to /content/csv_files/FY97.csv
Saved sheet 'FY98' to /content/csv_files/FY98.csv
Saved sheet 'FY99' to /content/csv_files/FY99.csv
Saved sheet 'FY00' to /content/csv_files/FY00.csv
Saved sheet 'FY01' to /content/csv_files/FY01.csv
Saved sheet 'FY02' to /content/csv_files/FY02.csv
Saved sheet 'FY03' to /content/csv_files/FY03.csv
Saved sheet 'FY04' to /content/csv_files/FY04.csv
Saved sheet 'FY05' to /content/csv_files/FY05.csv
Saved sheet 'FY06' to /content/csv_files/FY06.csv
Saved sheet 'FY07' to /content/csv_files/FY07.csv
Saved sheet 'FY08' to /content/csv_files/FY08.csv
Saved sheet 'FY09' to /content/csv_files/FY09.csv
Saved sheet 'FY10' to /content/csv_files/FY10.csv
Saved sheet 'FY11' to /content/csv_files/FY11.csv
Saved sheet 'FY12' to /content/csv_files/FY12.csv
Saved sheet 'FY13' to /content/csv_files/FY13.csv
Saved sheet 'FY14' to /content/csv_files/FY14.csv


ValueError: value_name (Quantity of U.S Visas Granted) cannot match an element in the DataFrame columns.

In [ ]:
# Directory where processed CSV files are saved
csv_directory = '/content/csv_files'

# List of processed CSV files
processed_csv_files = [f for f in os.listdir(csv_directory) if f.startswith('processed_') and f.endswith('.csv')]

# List to hold DataFrames for merging
dfs = []

# Read each processed CSV file and append it to the list
for csv_file in processed_csv_files:
    df = pd.read_csv(os.path.join(csv_directory, csv_file))
    dfs.append(df)

# Concatenate all DataFrames into one
merged_df = pd.concat(dfs, ignore_index=True)

# Save the merged DataFrame to a new CSV file
merged_csv_file = os.path.join(csv_directory, 'merged_data.csv')
merged_df.to_csv(merged_csv_file, index=False)

print(f"All processed CSV files merged and saved to {merged_csv_file}")

In [ ]:
visa_data = pd.read_csv('/content/csv_files/merged_data.csv')

In [ ]:
visa_data

In [ ]:
# Convert 'Year' column to numeric, coercing errors to NaN
visa_data['Year'] = pd.to_numeric(visa_data['Year'], errors='coerce')

# Check if any non-numeric values were present
print(visa_data['Year'].isna().sum(), "non-numeric values converted to NaN")

# Define a mapping from the codes to fiscal years

code_to_year_map = {
    97: 1997,
    98: 1998,
    99: 1999,
    0: 2000,
    1: 2001,
    2: 2002,
    3: 2003,
    4: 2004,
    5: 2005,
    6: 2006,
    7: 2007,
    8: 2008,
    9: 2009,
    10: 2010,
    11: 2011,
    12: 2012,
    13: 2013,
    14: 2014
}

# Apply the mapping to the 'Year' column to convert codes to actual years
visa_data['Year'] = visa_data['Year'].map(code_to_year_map)

In [ ]:
visa_data

In [ ]:
#Export CSV
visa_data.to_csv('visa_data.csv', index=False)